# Spring Boot Project Documentation Generator

In [1]:
!python --version

Python 3.12.8


In [2]:
import os

In [3]:
os.environ['CLOUD_ENV']='DEV51'

In [5]:
import paypal.aiplatform as paypal_aiplatform
import vertexai
from google.cloud import aiplatform
project_id = "dev52-test-apps-dpeaw"
location = "us-central1"
# Initialize SDK
paypal_aiplatform.init("paypal_genai_sdk_config.json")
vertexai.init(project=project_id, location=location)

In [44]:
from pathlib import Path
import re

In [40]:
def extract_methods(file_path):
    """Extracts method names and full method bodies from a given Java file."""
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # Match full method bodies in Java (simplified regex)
    method_pattern = re.findall(r'public\s+[\w<>\[\]]+\s+(\w+)\s*\(.*?\)\s*\{(.*?)\n\}', content, re.DOTALL)
    
    methods = {}
    for method_name, method_body in method_pattern:
        methods[method_name] = method_body.strip()
    
    return methods

In [41]:
def analyze_project_structure(project_path):
    """Scans the Spring Boot project and organizes files by type with full method details."""
    project_data = {'controllers': {}, 'services': {}, 'repositories': {}}
    for root, _, files in os.walk(project_path):
        for file in files:
            if file.endswith('.java'):
                file_path = Path(root) / file
                if 'controller' in root.lower():
                    project_data['controllers'][file] = extract_methods(file_path)
                elif 'service' in root.lower():
                    project_data['services'][file] = extract_methods(file_path)
                elif 'repository' in root.lower():
                    project_data['repositories'][file] = extract_methods(file_path)
    return project_data

In [42]:
def generate_documentation(project_data):
    
    prompt = f"""
    Generate detailed documentation for a Spring Boot project with the following structure:
    
    Controllers:
    {project_data['controllers']}
    
    Services:
    {project_data['services']}
    
    Repositories:
    {project_data['repositories']}
    
    Provide insights on:
    - Request handling in controllers
    - Business logic in services
    - Database interactions in repositories
    - Dependencies and method interactions
    """
    PROJECT_ID = "dev52-test-apps-dpeaw"
    vertexai.init(project=PROJECT_ID, location="us-central1")
    model = GenerativeModel(
        model_name="gemini-2.0-flash-thinking-exp-01-21", #"gemini-2.0-flash-exp", #"gemini-1.5-flash-002",
        system_instruction=[
            "You are a software documentation expert."
        ],
    )
    response = model.generate_content([prompt])
    return response.text

In [45]:
## Client Code
project_path = "/Users/kkailasnath/projects/spring/community/docker/docker-compose-springboot-mysql"
project_data = analyze_project_structure(project_path)

In [46]:
documentation = generate_documentation(project_data)

In [47]:
with open("SpringBoot_Documentation.md", "w", encoding="utf-8") as doc_file:
        doc_file.write(documentation)
print("Documentation generated: SpringBoot_Documentation.md")


Documentation generated: SpringBoot_Documentation.md
